### Imports

In [1]:
import os
import flywheel

import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf

from glob import glob
from os.path import join

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Helpers

In [2]:
def voxel_radius(radius):
    valid = list()
    for i in range(radius+1):
        for j in range(radius+1):
            for k in range(radius+1):
                if i == 0 and j == 0 and k == 0:
                    continue
                if i**2 + j**2 + k**2 > radius**2:
                    continue

                for parity in ([1, 1, 1], [-1, 1, 1], [1, -1, 1], [1, 1, -1],
                               [-1, -1, 1], [-1, 1, -1], [1, -1, -1], [-1, -1, -1]):
                    if ((i == 0 and parity[0] == -1) or
                        (j == 0 and parity[1] == -1) or
                        (k == 0 and parity[2] == -1)):
                        continue
                    valid.append({"x":  i * parity[0], "y":  j * parity[1], "z":  k * parity[2]})
    return valid

### Features

In [11]:
"Number of features:", (
    (len(voxel_radius(4)) * 2) + # 4 radius voxels (previous and next volume)
    (len(voxel_radius(4)) - len(voxel_radius(2))) + # 4 radius voxels not within 2 voxel radius for current volume
    2 + # time since (until) previous (next)
    1 + # time ratio between next voxels and previous
    2 + # time since last gonogo stimuli
    2 + # time since go/nogo bloack started
    6 + # time since faces stimuli
    6 + # time since faces block started
    1 + # time since keypress (any)
    2 + # gender, age
    3 + # task identifier (is_gonogo, is_(non)conscious)
    3 + # study identifier (conn, engage, rad)
    0
)

('Number of features:', 764)